In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
plt.style.use('ggplot')
# https://www.kaggle.com/fedesoriano/stroke-prediction-dataset 
df = pd.read_csv("..\\..\\data\\healthcare-dataset-stroke-data.csv")

In [2]:
# Generate the data samples: Original, Oversampled, Undersampled
from src.Resample import undersample_kmeans
from src.Resample import oversample

y = df[["stroke"]].copy()
X = df.copy()
X.drop(columns= ["id", "stroke"], inplace=True)

# HANDLE NAN VALUES HERE! Add method
df.dropna(inplace=True)

# Get over and undersampled data
undersampled = undersample_kmeans(df)
oversampled = oversample(df)

y_over = oversampled[["stroke"]].copy()
y_under = undersampled[["stroke"]].copy()

oversampled.drop(columns=["stroke"], inplace=True)
undersampled.drop(columns=["stroke"], inplace=True)

X_over = oversampled.copy()
X_under = undersampled.copy()

Original dataset samples per class Counter({0: 4700, 1: 209})
Resampled dataset samples per class Counter({0: 209, 1: 209})
Original dataset samples per class Counter({0: 4700, 1: 209})
Resampled dataset samples per class Counter({1: 4700, 0: 4700})


In [3]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVC

num_pipe = Pipeline([
    #('impute', SimpleImputer(strategy='median', copy=False)),
    ('minmax_scaler', MinMaxScaler(copy=False))
])

ordinal_pipe = Pipeline([
    ('one_hot', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

binary_pipe = Pipeline([
    ('label_encoder', OrdinalEncoder()),
])

# two pipelines combined in the column transformer
full_transform = ColumnTransformer([
    ("num", num_pipe, ["age", "avg_glucose_level", "bmi"]),
    ("ord", ordinal_pipe, ["gender", "work_type", "smoking_status"]),
    ("binary", binary_pipe, ["ever_married", "Residence_type"]),
])

full_pipeline = Pipeline([
    ('trf', full_transform),
    ('svm', SVC(random_state =0))
])

In [4]:
from sklearn.model_selection import train_test_split
# Split data

X_train, X_test, y_train, y_test = train_test_split(X_under, y_under, test_size=0.33, random_state=42, shuffle=True) # Set shuffle to true to have data of both labels in test set

In [5]:
full_pipeline.fit(X_train, y_train)
full_pipeline.score(X_test, y_test)

C:\Python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


0.7971014492753623